### Poisson Regression



In [1]:
library(foreign)
library(MASS)

Read data 

In [2]:
d=read.dta("needle_sharing.dta")

Preprocess data 

In [3]:
d=d[-which(d$sex=="Trans"),] # Remove rows where sex is Trans 
d=d[-which(d$hivstat==2),] # Remove rows where hiv status us 2

d=within(d,{ 
  id=NULL
  sex=factor(sex)
  sexabuse=factor(sexabuse)
  ethn=factor(ifelse(d$ethn=="AA"|d$ethn=="White"|d$ethn=="Hispanic",d$ethn,"Other"))
  shsyryn=NULL
  sqrtnivd=NULL 
  logshsyr=NULL 
  sqrtninj=NULL
  hivstat=factor(hivstat)
  polydrug=factor(polydrug)
  homeless=factor(homeless) 
  shsyr=NULL
})

d=na.omit(d)

Create base poisson model, stepwise model building, optimized by AIC 

In [4]:
base = glm(shared_syr~., data=d, family="poisson") # fit model on everything
final1 = stepAIC(base)

Start:  AIC=847.29
shared_syr ~ sex + ethn + age + dprsn_dx + sexabuse + hivstat + 
    hplsns + nivdu + polydrug + homeless

           Df Deviance     AIC
<none>          750.19  847.29
- dprsn_dx  1   757.11  852.20
- nivdu     1   757.28  852.38
- hplsns    1   760.31  855.41
- sexabuse  1   768.51  863.60
- age       1   776.03  871.13
- sex       1   783.83  878.93
- ethn      3   788.45  879.54
- polydrug  1   804.07  899.16
- hivstat   1   807.56  902.66
- homeless  1   920.29 1015.39


In [5]:
summary(final1)


Call:
glm(formula = shared_syr ~ sex + ethn + age + dprsn_dx + sexabuse + 
    hivstat + hplsns + nivdu + polydrug + homeless, family = "poisson", 
    data = d)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-6.4702  -1.9005  -1.0660  -0.5624  11.6138  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)   3.1598121  0.4899611   6.449 1.13e-10 ***
sexM         -0.9209149  0.1546092  -5.956 2.58e-09 ***
ethnHispanic -0.2829113  0.3314318  -0.854  0.39332    
ethnOther    -0.8262423  0.4368334  -1.891  0.05857 .  
ethnWhite    -1.1604997  0.1950254  -5.951 2.67e-09 ***
age          -0.0486900  0.0098113  -4.963 6.95e-07 ***
dprsn_dx      0.1465354  0.0547094   2.678  0.00740 ** 
sexabuse1    -1.2116955  0.3308155  -3.663  0.00025 ***
hivstat1     -2.6141802  0.4776113  -5.473 4.41e-08 ***
hplsns       -0.0521137  0.0168866  -3.086  0.00203 ** 
nivdu        -0.0020485  0.0008824  -2.322  0.02025 *  
polydrug1    -1.8488332  0.3142561  

In [8]:
# we accidentially forced linreg and nothing came back significant with a 65 disp parm

In parenthases we see the dispersion parameter is 1. This means the overdispersion parameter is forced to be 1. 

Null deviance - deviance of the model with just an intercept. 
Residual deviance - difference of deviance between our model and saturated model.

How can we test if this model is good as is?
hypothesis test of residual deviance. 
Why the dof 93? why 105 for dof of null deviance? 
93: n-(p+1) 

dim(data) = 106. So why dof 105? cuz it has a single param in the model, intercept only. 

why 93? 106- num parameters in model: 13. 

How to check if model fits the data well?
residual deviance is X2 distributed with 93 DOF.  Remember it rejects for large values. 
p-chi2

In [12]:
# Grab the deviance from the output
final1$deviance # residual deviance
final1$df.residual # degrees of freedom of residual deviance

[1] 750.1896

[1] 93

p value of hypothesis test for redisual deviance


In [11]:
1-pchisq(final1$deviance, final1$df.residual)

[1] 0

So is the model good? a p value of 0. the null hypothesis is that the model is good. It's overwhelmingly rejected.

remember X2 stat with n degrees of freedom has a mean of n and var of 2n. so when the dof is 92 and the value of the test statistic is 750 yo uare WAY away from a good fit, thats why the p value is 0. ***review

No.

What should we do?

Just to practice, if we really liked this model and didnt check the GOF. How would we interpret the coefs?

In [13]:
exp(final1$coef)

(Intercept)         sexM ethnHispanic    ethnOther    ethnWhite          age 
 23.56616676   0.39815462   0.75358660   0.43769091   0.31332958   0.95247635 
    dprsn_dx    sexabuse1     hivstat1       hplsns        nivdu    polydrug1 
  1.15781588   0.29769210   0.07322779   0.94922098   0.99795357   0.15742075 
   homeless1 
  9.30688928

expected number of syringes they shared with baseline cats
female baseline
ethnicity baseline AA

61% lower for males, everything else being equal
hispanic drug users 25% less than AA 
other "" 66% lower...
as you get older, each addtl year reduces number of shared needles by 5%

those that use multiple drugs have 84% decrease in number of shared needles. 

we kind of like it but it doesnt fit the data well. 

In [16]:
# We can test if overdispersion is present
library(AER)
dispersiontest(final1)


	Overdispersion test

data:  final1
z = 2.2554, p-value = 0.01206
alternative hypothesis: true dispersion is greater than 1
sample estimates:
dispersion 
  12.34147 


we reject the null hypothesis that there is no dispersion in favor of the aletenative that the dispersion parameter is greater than 1. in fact the disp parameter is 12.4
Our model had forced it to be 1. 

Lets refit the model, final1, with 

In [18]:
# final1# get the call and paste

Quasi poisson

In [19]:
final2 = glm(formula = shared_syr ~ sex + ethn + age + dprsn_dx + sexabuse + 
    hivstat + hplsns + nivdu + polydrug + homeless, family = "quasipoisson", 
    data = d)

In [20]:
summary(final2)


Call:
glm(formula = shared_syr ~ sex + ethn + age + dprsn_dx + sexabuse + 
    hivstat + hplsns + nivdu + polydrug + homeless, family = "quasipoisson", 
    data = d)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-6.4702  -1.9005  -1.0660  -0.5624  11.6138  

Coefficients:
              Estimate Std. Error t value Pr(>|t|)   
(Intercept)   3.159812   1.836732   1.720  0.08870 . 
sexM         -0.920915   0.579588  -1.589  0.11547   
ethnHispanic -0.282911   1.242448  -0.228  0.82038   
ethnOther    -0.826242   1.637570  -0.505  0.61507   
ethnWhite    -1.160500   0.731098  -1.587  0.11583   
age          -0.048690   0.036780  -1.324  0.18881   
dprsn_dx      0.146535   0.205091   0.714  0.47671   
sexabuse1    -1.211696   1.240138  -0.977  0.33107   
hivstat1     -2.614180   1.790436  -1.460  0.14764   
hplsns       -0.052114   0.063303  -0.823  0.41248   
nivdu        -0.002049   0.003308  -0.619  0.53722   
polydrug1    -1.848833   1.178061  -1.569  0.11995   
h

The effect sizes (beta estimates) didnt change. The variance that we forced is allowed to flow to be 12x bigger. this changed the significance. 

In [21]:
exp(2.23)

[1] 9.299866

With everything else constant, those who are homeless compared to the baseline of not homeless are 9.29 more likely to share syringes. 

### Negative Binomial Regression

In [23]:
final3 = glm.nb(formula = shared_syr ~ sex + ethn + age + dprsn_dx + sexabuse + 
    hivstat + hplsns + nivdu + polydrug + homeless,  
    data = d)

Warning message:
“glm.fit: algorithm did not converge”
Warning message:
“glm.fit: algorithm did not converge”
Warning message:
“glm.fit: algorithm did not converge”
Warning message:
“glm.fit: algorithm did not converge”
Warning message:
“glm.fit: algorithm did not converge”
Warning message:
“glm.fit: algorithm did not converge”
Warning message:
“glm.fit: algorithm did not converge”
Warning message:
“glm.fit: algorithm did not converge”
Warning message:
“glm.fit: algorithm did not converge”
Warning message:
“glm.fit: algorithm did not converge”
Warning message:
“glm.fit: algorithm did not converge”
Warning message:
“glm.fit: algorithm did not converge”
Warning message:
“glm.fit: algorithm did not converge”
Warning message:
“glm.fit: algorithm did not converge”
Warning message:
“glm.fit: algorithm did not converge”
Warning message:
“glm.fit: algorithm did not converge”
Warning message:
“glm.fit: algorithm did not converge”
Warning message:
“glm.fit: algorithm did not converge”
Warning me

warnings mean that we are overfitting

In [24]:
summary(final3)


Call:
glm.nb(formula = shared_syr ~ sex + ethn + age + dprsn_dx + sexabuse + 
    hivstat + hplsns + nivdu + polydrug + homeless, data = d, 
    init.theta = 0.09180651863, link = log)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.1426  -0.7400  -0.5057  -0.2215   1.9104  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)   
(Intercept)   4.198740   2.449838   1.714  0.08655 . 
sexM         -0.561843   0.932189  -0.603  0.54670   
ethnHispanic  1.936667   1.739433   1.113  0.26554   
ethnOther    -2.104401   2.276698  -0.924  0.35532   
ethnWhite    -0.204575   0.971663  -0.211  0.83325   
age          -0.096842   0.047922  -2.021  0.04330 * 
dprsn_dx      0.230686   0.257752   0.895  0.37079   
sexabuse1    -2.124947   1.292540  -1.644  0.10017   
hivstat1     -2.364185   1.654765  -1.429  0.15309   
hplsns        0.040089   0.079165   0.506  0.61258   
nivdu        -0.007208   0.006500  -1.109  0.26747   
polydrug1    -3.627161   1.381279  -2

if you compare the coefs
obviously pois and qpois had the same estimates with different p values and std errors.

Comparing coefs from the 3 models

In [27]:
cbind(final1$coef, final2$coef, final3$coef)

(Intercept),3.159812072,3.159812072,4.198740178
sexM,-0.920914854,-0.920914854,-0.561842743
ethnHispanic,-0.282911338,-0.282911338,1.936666577
ethnOther,-0.826242306,-0.826242306,-2.104400761
ethnWhite,-1.160499673,-1.160499673,-0.204574533
age,-0.048690004,-0.048690004,-0.096842178
dprsn_dx,0.146535370,0.146535370,0.230686226
sexabuse1,-1.211695533,-1.211695533,-2.124947301
hivstat1,-2.614180235,-2.614180235,-2.364185095
hplsns,-0.052113655,-0.052113655,0.040089063
nivdu,-0.002048526,-0.002048526,-0.007208498


Wyt about hte first 2 cols? Pois, qpois, thirs col nb

relative risk ratios in pois regren.

next time:
different people are followed over diff periods of time: you have to force a variable in the model called an offset.